<a href="https://colab.research.google.com/github/chineidu/NLP-Tutorial/blob/main/notebook/06_Transformers/NLP-With-Transformers/02-NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiingual Name Entity Recognition



In [1]:
!pip install -q transformers accelerate datasets mlxtend watermark rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
%load_ext watermark
%watermark -v -p numpy,pandas,polars,mlxtend,transformers --conda

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy       : 1.26.4
pandas      : 2.1.4
polars      : 0.20.2
mlxtend     : 0.23.1
transformers: 4.42.4

conda environment: n/a



In [3]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "info": "#76FF7B",
        "warning": "#FBDDFE",
        "error": "#FF0000",
    }
)
console = Console(theme=custom_theme)

# Visualization
import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")


# auto reload imports# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "info": "#76FF7B",
        "warning": "#FBDDFE",
        "error": "#FF0000",
    }
)
console = Console(theme=custom_theme)

# Visualization
import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(500)

warnings.filterwarnings("ignore")


# # Black code formatter (Optional)
# %load_ext lab_black

# # auto reload imports
# %load_ext autoreload
# %autoreload 2

In [4]:
from datasets import get_dataset_config_names, load_dataset


xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

XTREME has 183 configurations


In [5]:
# Select the config that starts with `PAN`
pan_subsets: list[str] = [s for s in xtreme_subsets if s.startswith("PAN")]
print(pan_subsets[:3])

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']


In [6]:
from datasets import DatasetDict


# Load English subset
en_dataset: DatasetDict = load_dataset("xtreme", name="PAN-X.en")
en_dataset

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [7]:
# Shuffle
en_dataset['train'].shuffle(seed=123)

# Select a specific number of records
print(en_dataset['train'].select(range(10)))

# Check the number pf records
print(f"Number of records: {len(en_dataset['train']):,}")

# OR
print(f"Number of records: {en_dataset['train'].num_rows:,}")

Dataset({
    features: ['tokens', 'ner_tags', 'langs'],
    num_rows: 10
})
Number of records: 20,000
Number of records: 20,000


#### Coment

- To create a realistic Swiss corpus, we'll sample PAN-X corpora in German (62.9%), French (22.9%), Italian (8.4%), and English (5.9%) according to their spoken proportions in Switzerland.
- This will create a language imbalance simulating real-world datasets.
- We'll use a Python `defaultdict` to store language codes and corresponding PAN-X corpora.

In [8]:
from collections import defaultdict


seed: int = 123
langs: list[str] = ["de", "fr", "it", "en"]
fracs: list[float] = [0.629, 0.229, 0.084, 0.059]
panx_ch: defaultdict = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # Load dataset
    ds: DatasetDict = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # Shuffle and downsample each split according to spoken proportion
    for split in ds:
      num_rows: int = ds[split].num_rows
      panx_ch[lang][split] = ds[split].shuffle(seed=seed).select(range(int(frac * num_rows)))


Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
panx_ch

defaultdict(datasets.dataset_dict.DatasetDict,
            {'de': DatasetDict({
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 12580
                 })
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
             }),
             'fr': DatasetDict({
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 4580
                 })
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'la

In [10]:
for key, val in panx_ch.items():
  print(val)
  break

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 12580
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 6290
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 6290
    })
})


In [11]:
# Number of training examples per language
res: dict[str, int] = {key:val["train"].num_rows for key, val in panx_ch.items()}
pl.DataFrame(res)


de,fr,it,en
i64,i64,i64,i64
12580,4580,1680,1180


In [12]:
# Inspect one of the languages
sample = panx_ch["en"]["train"][0]
sample

{'tokens': ['Collin', 'Peterson', '(', 'D-MN', ')'],
 'ner_tags': [1, 2, 0, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en']}

In [13]:
print(panx_ch["en"]["train"].features)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None), 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


In [14]:
ner_tags = panx_ch["en"]["train"].features["ner_tags"]
tags = ner_tags.feature
print(f"{ner_tags = }")
print(f"{tags = }")

ner_tags = Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
tags = ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


In [15]:
for idx, _ in enumerate(tags.names):
  console.print(f"{idx} ==> {tags.int2str(idx)}")

0 ==> O

1 ==> B-PER

2 ==> I-PER

3 ==> B-ORG

4 ==> I-ORG

5 ==> B-LOC

6 ==> I-LOC

In [16]:
pprint(panx_ch["en"]["train"][10])

{'langs': ['en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en',
           'en'],
 'ner_tags': [0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
 'tokens': ['Tries',
            ':',
            'Todd',
            'Carney',
            ',',
            'Ricky',
            'Leutele',
            ',',
            'Michael',
            'Gordon',
            ',',
            'Jeff',
            'Robson']}


In [17]:
def create_tag_names(batch: DatasetDict) -> dict[str, Any]:
  return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}


panx_de: DatasetDict = panx_ch["de"].map(create_tag_names)

panx_de["train"][5]

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

{'tokens': ['Die',
  'meisten',
  'seiner',
  'Gemälde',
  'sind',
  'im',
  'Statens',
  'Museum',
  'for',
  'Kunst',
  'zu',
  'sehen',
  '.'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0],
 'langs': ['de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de'],
 'ner_tags_str': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O',
  'O']}

In [18]:
pl.DataFrame(panx_de["train"].select_columns(['tokens', 'ner_tags_str'])[5]).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12
str,str,str,str,str,str,str,str,str,str,str,str,str
"""Die""","""meisten""","""seiner""","""Gemälde""","""sind""","""im""","""Statens""","""Museum""","""for""","""Kunst""","""zu""","""sehen""","""."""
"""O""","""O""","""O""","""O""","""O""","""O""","""B-ORG""","""I-ORG""","""I-ORG""","""I-ORG""","""O""","""O""","""O"""


In [19]:
# Check the distribution of all the tags
from collections import Counter


splits_freq = defaultdict(Counter)

for split, ds in panx_de.items():
  for row in ds["ner_tags_str"]:
    for tag in row:
      # Focus on the `beginning` tags
      if tag.startswith("B"):
        tag_type: str = tag.split("-")[1]
        splits_freq[split][tag_type] += 1

In [29]:
# The tags are roughly equally distributed
console.print(f"{splits_freq = }")

pl.DataFrame(splits_freq).to_pandas()

splits_freq = defaultdict(<class 'collections.Counter'>, {'train': Counter({'LOC': 6039, 'PER': 5862, 'ORG': 
5413}), 'validation': Counter({'LOC': 3095, 'PER': 2837, 'ORG': 2721}), 'test': Counter({'LOC': 3145, 'PER': 2962, 
'ORG': 2620})})

,train,validation,test
0,"{'PER': 5862, 'LOC': 6039, 'ORG': 5413}","{'PER': 2837, 'LOC': 3095, 'ORG': 2721}","{'ORG': 2620, 'PER': 2962, 'LOC': 3145}"


<br>

## Multilingual Transformers

- Multilingual transformers are trained on multilingual corpora and they can generalize well across languages for various tasks.

- This approach can outperform monolingual models for cross-lingual transfer, avoiding the need to train separate models for each language.

- `CoNLL-2002` and `CoNLL-2003` are commonly used benchmarks for NER in these languages.

- Multilingual transformer models are usually evaluated in three different ways:
  - `en`: Fine-tune on the English training data and then evaluate on each language's test set.
  - `each`: Fine-tune and evaluate on monolingual test data to measure per-language performance.
  - `all`: Fine-tune on all the training data to evaluate on all on each languag's test set.

  <br>

- [XLM-RoBERTa](https://huggingface.co/docs/transformers/en/
model_doc/xlm-roberta) (`XLM-R`) model will be used.

- XLM-RoBERTa (XLM-R) is a multilingual transformer model trained on a massive dataset of text and code in 100 different languages.
  
- It can be used for a variety of tasks, including:

  - Text classification: Sentiment analysis, topic classification, etc.
  - Question answering: Extracting answers to questions from a given text.
  - Translation: Translating text from one language to another.
  - Named entity recognition: Identifying named entities in text, such as people, organizations, and locations.

- XLM-R achieves state-of-the-art results on many cross-lingual benchmarks, and it is a powerful tool for natural language processing in multiple languages.

#### Tokenization: SentencePiece



In [31]:
from transformers import AutoTokenizer


bert_model_name: str = "bert-base-cased"
xlmr_model_name: str = "xlm-roberta-base"

bert_tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [32]:
text: str = "Jack Sparrow loves New York!"

bert_tokens: list[str] = bert_tokenizer(text).tokens()
xlmr_tokens: list[str] = xlmr_tokenizer(text).tokens()

print(f"{bert_tokens = }")
print(f"{xlmr_tokens = }")

bert_tokens = ['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']
xlmr_tokens = ['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']


### Tokenization Pipeline

<br>

[![image.png](https://i.postimg.cc/zvJ0SrZz/image.png)](https://postimg.cc/dkx2wgVp)

<br>

- `Normalization`: it involves cleaning raw text by removing whitespace, accents, and standardizing Unicode characters. It also includes lowercasing to reduce vocabulary size. After normalization, our example string becomes "jack sparrow loves new york!".

- `Pretokenization`: it splits text into words for easier tokenization. For English and similar languages, this is simple. For languages like Chinese, it's more complex and might require language-specific libraries.

- `Tokenizer model`: it splits words into subwords to reduce vocabulary size and out-of-vocabulary tokens. This is done using algorithms like BPE, Unigram, and WordPiece. For example, "jack sparrow" might become "[jack, spa, rrow]".

- `Postprocessing`: it's the final step in tokenization, where additional tokens (like [CLS] and [SEP]) are added to the beginning and end of the token sequence to prepare it for input into a model like BERT.


<hr><br>

#### SentencePiece Tokenizer

- The SentencePiece tokenizer is a flexible subword segmentation method that handles multilingual text effectively by preserving whitespace and using Unicode characters.

- It avoids ambiguities in detokenization and can be used for various languages without relying on language-specific pretokenizers.

### Transformers For Classification Vs NER

**1.) Classification**

[![image.png](https://i.postimg.cc/Mph53R2r/image.png)](https://postimg.cc/tsNFYsgh)

<br>

**2.) Named Entity Recognition**

[![image.png](https://i.postimg.cc/26YwrD5Q/image.png)](https://postimg.cc/rdhWN7xs)

<br>

- In `token classification`, assign the label (e.g., B-PER) to the first subword ("Chr") and ignore subsequent subwords ("##ista"). This convention follows the BERT paper and maintains the IOB2 format. Postprocessing can propagate the label to all subwords.


### Anatomy Transformers Model Class

- The Transformers API organizes models by task, but this can be limiting.
- If a desired task model doesn't exist, it can be difficult to implement a custom model.
- However, Transformers provides tools to load pretrained weights and use task-specific helper functions, making it easier to create custom models.

### Bodies And Heads

- Transformers are versatile due to their `body-head` structure.
- The `head` is task-specific, while the `body` is task-agnostic.
- The body includes embeddings and transformer layers. The Transformers code reflects this structure with classes like BertModel and GPT2Model that return hidden states.
- Task-specific models such as `BertForMaskedLM` or `BertForSequenceClassification` use the base model and add the necessary head on top of the hidden states.


<hr><br>

### Creating Custom Model For Token Classification

- Building a custom token classification head for XLM-R is similar to RoBERTa and an existing class, XLMRobertaForTokenClassification, can be used directly for this task.
- However, going through the exercise of building a custom model for educational purposes.

- We need a data structure for our XLM-R NER tagger.
- It will have a configuration object and a forward function.

In [33]:
from torch import Tensor
import torch.nn as nn
from transformers import XLMRobertaConfig, XLMRobertaModel
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel


class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    class_config = XLMRobertaConfig

    def __init__(self, config) -> None:
        super().__init__(config)
        self.num_labels = config.num_labels

        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False)

        # Setup classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Load and initialize weights
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs
    ):
        # Get the encoder representations using model body
        outputs: Tensor = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs
        )

        # Apply classifier to encoder reps
        sequence_output: Tensor = self.dropout(outputs[0])
        logits: Tensor = self.classifier(sequence_output)

        # Calculate losses
        loss: Optional[Tensor] = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Return model output object
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

#### Comment

- The config_class ensures standard XLM-R settings.
- The super() method calls the initialization function of the RobertaPreTrainedModel class.
- The model body is extended with a classification head.
- The forward() method defines the model's behavior.
- The hidden state is fed through the dropout and classification layers.
- The loss is calculated if labels are provided.
- The outputs are wrapped in a TokenClassifierOutput object.
- This custom transformer model inherits from a PreTrainedModel, allowing access to useful Transformer utilities.

<br>

### Loading A Custom Model
